In [2]:
import torch
import torch.nn as nn

torch.manual_seed(0)

In [3]:
example = torch.rand(5,3)


In [4]:
available_device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device: {available_device}")

Device: mps


In [5]:
device = torch.device(available_device)

In [6]:
x = torch.randn(5,3, requires_grad=True)
y = x + 2
print(y)

tensor([[2.6035, 2.8110, 1.9549],
        [2.8797, 3.0482, 1.9555],
        [1.2771, 4.8663, 1.4345],
        [2.1604, 1.9746, 3.0739],
        [4.2628, 1.0825, 1.7749]], grad_fn=<AddBackward0>)


In [7]:
# Slicing example
example = torch.rand(3,3, requires_grad=True).to(device)
print(example)
print(example[:2, :])

tensor([[0.7932, 0.2783, 0.4820],
        [0.8198, 0.9971, 0.6984],
        [0.5675, 0.8352, 0.2056]], device='mps:0', grad_fn=<ToCopyBackward0>)
tensor([[0.7932, 0.2783, 0.4820],
        [0.8198, 0.9971, 0.6984]], device='mps:0', grad_fn=<SliceBackward0>)


In [8]:
example.get_device()

0

In [9]:
# Preventing gradient history
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)
    weights.grad.zero_()
    
    
    
    

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [10]:
# Backprop example 
x = torch.tensor(1)
y = torch.tensor(2)

w = torch.tensor(1.0, requires_grad=True)
loss = torch.pow((x * w - y),2)

print(loss)

tensor(1., grad_fn=<PowBackward0>)


In [11]:
loss.backward()
print(w.grad)

tensor(-2.)


In [12]:
import numpy as np


In [13]:
# Simple manual ml algorithm
# f(x) = 3 * x -> this is the target

x = np.array([1,2,3,4], dtype=np.float32)
y = np.array([3,6,9,12], dtype=np.float32)

weights = 0.0

# Forward pass
def forward(features):
    return weights * features

def loss(prediction, y_true):
    return ((prediction - y_true)**2).mean()

# gradient = 1/N * 2*x * (prediction-y_true)
def gradient(prediction, features, y_true):
    return np.dot(2*features, prediction -y_true).mean()

In [14]:
# Training loop 
learning_rate = 0.01
print(f"Prediction before training: {forward(5):.3f}")

for epoch in range(100):
    y_pred = forward(x)
    mse = loss(y_pred, y)
    dw = gradient(y_pred,x,y)
    weights -= learning_rate * dw

print(f"Prediction after training: {forward(5):.3f}")

Prediction before training: 0.000
Prediction after training: 15.000


In [15]:
# Using pytorch 

# Forward pass



x =  torch.tensor([1,2,3], dtype=torch.float32)
y = torch.tensor([2,4,6], dtype=torch.float32)
weights = torch.tensor(0.0, dtype=torch.float32 ,requires_grad=True)


def forward(features):
    return weights * features

def loss(prediction, y_true):
    return ((prediction - y_true)**2).mean()


for epoch in range(500):
    y_pred = forward(x)
    
    l = loss(y_pred, y)
    l.backward()
    
    with torch.no_grad():
        weights -= learning_rate * weights.grad
    
    weights.grad.zero_()

print(f"Prediction after training: {forward(5):.3f}")

Prediction after training: 10.000


In [29]:
# Simple pytorch model
# 1. Define the model (inputs , output, forward pass)
# 2. Construct the loss and optimizer
# 3. Training loop
#       - forward pass
#       - backward pass
#       - update the weights

class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression,self).__init__()
        
        self.fc1 = nn.Linear(input_size, output_size)
        
        
    def forward(self, inputs: torch.Tensor):
        return self.fc1(inputs)
    

model = LinearRegression(1,1).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [34]:
# Training loop for simple model
x = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32).to(device)
y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32).to(device)
x_test = torch.tensor([[5]], dtype=torch.float32).to(device)
n_samples, n_features = x.shape

for epoch in range(50):
    
    # Forward pass
    prediction = model(x)
    
    # Calculate the loss
    loss = criterion(prediction, y.to(device))
    
    # Backpropagation
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    # Zero out the gradients to prevent accumulation
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        [w, b] = model.parameters()
        # print(w)
        print(f" weights: {w[0][0]:.3f} | loss: {loss:.2f}")

print(f"Prediction after training: {model(x_test).item():.3f}")  

 weights: 2.000 | loss: 0.00
 weights: 2.000 | loss: 0.00
 weights: 2.000 | loss: 0.00
 weights: 2.000 | loss: 0.00
 weights: 2.000 | loss: 0.00
Prediction after training: 10.000


In [36]:
# Logistic Regression with Pytorch
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0 Prepare the dataset 
breast_cancer_data = datasets.load_breast_cancer()
X , y = breast_cancer_data.data , breast_cancer_data.target

n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30, shuffle=True)

# NB: usually, only the features are normalized or scaled.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test= torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)        

In [39]:
# 1. Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression,self).__init__()

        self.fc1 = nn.Linear(input_size, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 8)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(8,1)


    def forward(self, inputs: torch.Tensor):
        inputs = self.fc1(inputs)
        inputs = self.relu1(inputs)
        inputs = self.fc2(inputs)
        inputs = self.relu2(inputs)
        inputs = self.fc3(inputs)
        return torch.sigmoid(inputs)

model = LogisticRegression(n_features).to(device)

# 3. loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 4. Training loop 
for epoch in range(500):


    prediction =  model(X_train.to(device)) # Forward pass
    loss = criterion(prediction, y_train.to(device)) # Calculate the loss
    loss.backward()     # Backprop
    optimizer.step() # Updated the params
    optimizer.zero_grad() # Zero out the gradients to prevent gradient accumulation


In [40]:
with torch.no_grad():
    y_pred = model(X_test.to(device))
    output = y_pred.round()
    acc = output.eq(y_test.to(device)).sum() /float(y_test.size(0))
    print(f"Accuracy: {acc}")

Accuracy: 0.9912280440330505
